In [1]:
from random import randint
from datetime import date, datetime
import pandas as pd
from pathlib import Path

In [2]:
users = ['Craig', 'Gary', 'Sean', 'Thomas']

users

['Craig', 'Gary', 'Sean', 'Thomas']

In [3]:
def get_initial_batman_and_robin(users):
    choices = users.copy()
    batman = choices[randint(0, len(choices) - 1)]
    choices.remove(batman)
    robin = choices[randint(0, len(choices) - 1)]

    return (batman, robin)

get_initial_batman_and_robin(users)

('Craig', 'Sean')

In [4]:
def get_batman(users, batman, robin):
    choices = users.copy()
    choices.remove(batman)
    choices.remove(robin)
    
    return choices[randint(0, len(choices) - 1)]

get_batman(users, 'Craig', 'Sean')

'Thomas'

In [5]:
def get_champs(candidates, batman = None, robin = None):
    choices = candidates.copy()

    if batman != None:
        choices.remove(batman)

    if robin != None:
        choices.remove(robin)

    batman = choices[randint(0, len(choices) - 1)]
    choices.remove(batman)
    robin = choices[randint(0, len(choices) - 1)]
    return (batman, robin)

print(get_champs(users))
print(get_champs(users, 'Thomas'))
print(get_champs(users, robin='Sean'))
print(get_champs(users, batman='Craig'))
print(get_champs(users, 'Gary', 'Thomas'))

('Gary', 'Craig')
('Craig', 'Sean')
('Thomas', 'Craig')
('Thomas', 'Sean')
('Craig', 'Sean')


In [6]:
users

['Craig', 'Gary', 'Sean', 'Thomas']

In [7]:
def get_date_range(start_date, end_date):
    dates = pd.date_range(start_date, end_date, freq='W-MON')
    return dates
    # return [i.strftime('%Y-%m-%d') for i in dates]
    
start_date = date(2021, 1, 1)
end_date = date(2021, 12, 31)
dates = get_date_range(start_date, end_date)

print(dates[:5])

DatetimeIndex(['2021-01-04', '2021-01-11', '2021-01-18', '2021-01-25',
               '2021-02-01'],
              dtype='datetime64[ns]', freq='W-MON')


In [8]:
def populate_users(dates, users):
    data = []
    for i in range(len(dates)):
        if i == 0:
            # batman, robin = get_initial_batman_and_robin(users)
            batman, robin = get_champs(users)
        else:
            robin = data[i - 1]['Batman']
            batman = get_champs(users, robin=robin)[0]
        data.append({
            'Weekly date': dates[i],
            'Batman': batman,
            'Robin': robin,
        })
        
    return pd.DataFrame(data)

rota = pd.DataFrame(populate_users(dates, users))
rota.head(5)

,Weekly date,Batman,Robin
0,2021-01-04,Thomas,Gary
1,2021-01-11,Sean,Thomas
2,2021-01-18,Craig,Sean
3,2021-01-25,Thomas,Craig
4,2021-02-01,Gary,Thomas


In [19]:
import calendar
from datetime import timedelta

def get_last_day_of_the_month(dto):
    weekday, days = calendar.monthrange(dto.year, dto.month)
    return date(dto.year, dto.month, days)

last_day = get_last_day_of_the_month(today)
print(f'Last Date in the month: {last_day}')

def is_first_day_monday(dto):
    weekday, days = calendar.monthrange(dto.year, dto.month)
    return weekday == 0

print(f'Is the first day a Monday? {is_first_day_monday(today)}')

def is_draw_week(start_date, end_date):
    # Wednesday must be the 1st at the very latest

def get_monday(dto):
    if 0 == dto.weekday():
        return dto
    return dto - timedelta(days=-dto.weekday(), weeks=1)

def get_friday(dto):
    if 4 == dto.weekday():
        return dto
    return dto - timedelta(days=dto.weekday()) + timedelta(days=4, weeks=-1)

today = date.today()
monday = get_monday(today)
friday = get_friday(today)

print(f'Today={today} ({calendar.day_name[today.weekday()]}), Last Monday={monday}, Next Friday={friday}')

Last Date in the month: 2021-04-30
Is the first day a Monday? False
Today=2021-04-16 (Friday), Last Monday=2021-04-13, Next Friday=2021-04-16


In [11]:
rota['End of Week'] = rota['Weekly date'] + pd.offsets.Week(weekday=4)
rota['Weekday start'] = rota['Weekly date'].dt.day_name()
rota['Weekday end'] = rota['End of Week'].dt.day_name()
rota['Draw Week'] = rota['Weekly date']
rota = rota[['Weekly date', 'Weekday start', 'End of Week', 'Weekday end', 'Draw Week', 'Batman', 'Robin']]
rota.head()

,Weekly date,Weekday start,End of Week,Weekday end,Draw Week,Batman,Robin
0,2021-01-04,Monday,2021-01-08,Friday,2021-01-04,Thomas,Gary
1,2021-01-11,Monday,2021-01-15,Friday,2021-01-11,Sean,Thomas
2,2021-01-18,Monday,2021-01-22,Friday,2021-01-18,Craig,Sean
3,2021-01-25,Monday,2021-01-29,Friday,2021-01-25,Thomas,Craig
4,2021-02-01,Monday,2021-02-05,Friday,2021-02-01,Gary,Thomas


In [12]:
data_file = Path('data', 'routa.csv')
df = pd.read_csv(data_file)
df.head(5)


,Weekly Dates,Batman Support,Robin Support
0,04/01/2021,Craig,Thomas
1,11/01/2021,Thomas,Gary
2,18/01/2021,Gary,Sean
3,25/01/2021,Sean,Thomas
4,01/02/2021,Thomas,Gary


In [13]:
rota.groupby(['Batman'])['Weekly date'].count()

Batman
Craig      9
Gary      11
Sean      13
Thomas    19
Name: Weekly date, dtype: int64